In [1]:
%matplotlib inline
#mpld3.enable_notebook()
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from datetime import datetime
import xarray as xr
from astropy.io import ascii
import pytz
# OS interaction
import sys
import os
import glob
import seaborn as sns
sns.set_context("talk",font_scale=1.5)
sns.set_style('whitegrid')

# User config

In [2]:
# Paths to user files
data_dir = os.path.normpath(r'F:\Work\e\Data\Obs\Canada_Project_Sites\CSAS_data') # Where to store data on local computer

# Create paths

In [3]:
AB_recent = os.path.join(data_dir,'AB_recent','netcdf','AB_NRT.nc')
AB_POR    = os.path.join(data_dir,'AB_POR','netcdf','AB_SWE_SD_NRT.nc')
BC_CWY    = os.path.join(data_dir,'BC_NRT','netcdf','BC_NRT.nc') # Current Water Year (CWR)

In [4]:
merged_dir = os.path.join(data_dir,'merged')
# Make if does not exist
if not os.path.exists(merged_dir):
    os.makedirs(merged_dir)
netcdf_file_out = os.path.join(merged_dir, 'Hourly_Merged.nc')

# Merge Networks

In [5]:
ds_AB_recent = xr.open_dataset(AB_recent)
ds_AB_POR    = xr.open_dataset(AB_POR)
ds_BC_CWR    = xr.open_dataset(BC_CWY)

In [6]:
ds_AB_POR

<xarray.Dataset>
Dimensions:       (Time_UTC: 315320, staID: 13)
Coordinates:
  * staID         (staID) object '05AA809' '05AA817' '05AD803' '05BB803' ...
  * Time_UTC      (Time_UTC) datetime64[ns] 1983-12-07T04:40:00 ...
    station_name  (staID) object 'Gardiner Creek' 'South Racehorse Creek' ...
    Lat           (staID) float64 49.36 49.78 49.03 51.08 50.63 50.82 50.17 ...
    Lon           (staID) float64 -114.5 -114.6 -114.1 -115.8 -115.3 -115.0 ...
    Elevation     (staID) float64 1.97e+03 1.92e+03 1.761e+03 2.23e+03 ...
    network       (staID) object 'environmentAlberta' 'environmentAlberta' ...
Data variables:
    SWE           (staID, Time_UTC) float64 nan nan nan nan nan nan nan nan ...
    Snowdepth     (staID, Time_UTC) float64 nan nan nan nan nan nan nan nan ...

In [7]:
# Merge AB data together (hist to midnight, and last few days)
###### This step drops coords info so leaving out most recent data for now
# ds_AB_mrg = ds_AB_POR.combine_first(ds_AB_recent)
ds_AB_mrg = ds_AB_POR

In [8]:
ds_merged = xr.merge([ds_AB_mrg,ds_BC_CWR])

In [9]:
ds_merged

<xarray.Dataset>
Dimensions:         (Time_UTC: 315344, staID: 85)
Coordinates:
  * Time_UTC        (Time_UTC) datetime64[ns] 1983-12-07T04:40:00 ...
  * staID           (staID) object '05AA809' '05AA817' '05AD803' '05BB803' ...
    station_name    (staID) object 'Gardiner Creek' 'South Racehorse Creek' ...
    Lat             (staID) float64 49.36 49.78 49.03 51.08 50.63 50.82 ...
    Lon             (staID) float64 -114.5 -114.6 -114.1 -115.8 -115.3 ...
    Elevation       (staID) float64 1.97e+03 1.92e+03 1.761e+03 2.23e+03 ...
    network         (staID) object 'environmentAlberta' 'environmentAlberta' ...
Data variables:
    SWE             (staID, Time_UTC) float64 nan nan nan nan nan nan nan ...
    Snowdepth       (staID, Time_UTC) float64 nan nan nan nan nan nan nan ...
    AirTemperature  (staID, Time_UTC) float64 nan nan nan nan nan nan nan ...
    Precipitation   (staID, Time_UTC) float64 nan nan nan nan nan nan nan ...

In [10]:
# Save as netcdf file
ds_merged.to_netcdf(netcdf_file_out)

In [11]:
set(ds_merged.network.values)

{'bcRiverForecastCenter', 'environmentAlberta'}